# Уровень лояльности телекоммуникационной компании

<b>Цель исследования:</b> Определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

<b>Ход исследования:</b> Данные берем из базы: telecomm_csi.db. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. Поэтому с помощью запросов выгрузим необходимые нам поля. И на основании полученного файла проведем исследование.

<b>Этапы исследования:</b>

- Подключение к базе;
- Выгрузка данных;
- Создание дашборда в Tableau;
- Анализ данных и основные выводы;
- Презентация.

<b>Описание данных:</b>

Компания провела опрос на основании которого нам и нужно провести анализ.

Чтобы оценить результаты опроса, оценки разделим на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

<b>Таблица user:</b>

- user_id - Идентификатор клиента, первичный ключ таблицы
- lt_day - Количество дней «жизни» клиента
- age - Возраст клиента в годах
- gender_segment - Пол клиента (1 – женщина, 0 – мужчина)
- os_name - Тип операционной системы
- cpe_type_name - Тип устройства
- location_id - Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
- age_gr_id - Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
- tr_gr_id - Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
- lt_gr_id - Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
- nps_score - Оценка клиента в NPS-опросе (от 1 до 10)

<b>Таблица location:</b>

- location_id - Идентификатор записи, первичный ключ
- country - Страна
- city - Город

<b>Таблица age_segment:</b>

- age_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

<b>Таблица traffic_segment:</b>

- tr_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

<b>Таблица lifetime_segment:</b>

- lt_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

## Шаг 1. Подключение к базе

In [ ]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [ ]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/content/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Шаг 2. Выгрузка данных

In [ ]:
query = """
WITH
u AS (SELECT user_id,
             lt_day,
             CASE
                WHEN lt_day > 365 THEN 1
                ELSE 0
             END AS is_new,
             age,
             CAST(gender_segment AS varchar) AS gender_segment,
             os_name,
             cpe_type_name,
             location_id,
             age_gr_id,
             tr_gr_id,
             lt_gr_id
      FROM user),
l AS (SELECT country,
             city,
             location_id
      FROM location),
a AS (SELECT title AS age_segment,
             age_gr_id
      FROM age_segment),
t AS (SELECT title AS traffic_segment,
             tr_gr_id
      FROM traffic_segment),
lt AS (SELECT title AS lifetime_segment,
              lt_gr_id
      FROM lifetime_segment),
uc AS (SELECT nps_score,
              CASE
                WHEN nps_score >= 9 THEN 'cторонники'
                WHEN nps_score >= 7 AND nps_score < 9 THEN 'нейтралы'
                ELSE 'критики'
              END AS nps_group,
              user_id
      FROM user)
SELECT u.user_id,
       u.lt_day,
       u.is_new,
       u.age,
       u.gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       a.age_segment,
       t.traffic_segment,
       lt.lifetime_segment,
       uc.nps_score,
       uc.nps_group
FROM u LEFT OUTER JOIN l ON u.location_id=l.location_id
LEFT OUTER JOIN a ON u.age_gr_id=a.age_gr_id
LEFT OUTER JOIN t ON u.tr_gr_id=t.tr_gr_id
LEFT OUTER JOIN lt ON u.lt_gr_id=lt.lt_gr_id
LEFT OUTER JOIN uc ON u.user_id=uc.user_id;
"""

После написания запроса проверяем чтобы все данные были корректные. Так же выведем info().

In [ ]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,1,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,cторонники
1,A001WF,2344,1,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,cторонники
2,A003Q7,467,1,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,cторонники


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  int64  
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(3), object(10)
memory usage: 53.7+ MB


In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Видим что все данные подтянулись, а так же gender_segment отображается в нужном формате. Скачиваем файл и идем в Tableau

Ссылка на проект в Tableau:
https://public.tableau.com/app/profile/eudgene.kor/viz/Prefabricated_project_2/NPS-_4?publish=yes